### Get credentials

In [162]:
import os
import requests
from dotenv import load_dotenv
from googleapiclient.discovery import build

# Load API keys from .env
load_dotenv()
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")
YT_TRANSCRIPT_API_KEY = os.getenv("YT_TRANSCRIPT_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ['MallocStackLogging'] = '0'
os.environ['MallocStackLoggingNoCompact'] = '0'

## Youtube API

In [8]:
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

try:
    request = youtube.channels().list(
        part="snippet",
        id="UC_x5XG1OV2P6uZZ5FSM9Ttw"  # Example: Google Developers
    )
    response = request.execute()
    print("YouTube API key is valid!")
    print("Channel title:", response['items'][0]['snippet']['title'])
except Exception as e:
    print("YouTube API key is INVALID or there is another error.")
    print(e)

YouTube API key is valid!
Channel title: Google for Developers


### Get Channel ID from name

In [13]:

# Use the handle or channel name as the query
query = "theunapologeticmanpodcast"

request = youtube.search().list(
    part="snippet",
    q=query,
    type="channel",
    maxResults=1
)
response = request.execute()
print(response)

# Extract channel ID
if response['items']:
    channel_id = response['items'][0]['snippet']['channelId']
    print("Channel ID:", channel_id)
else:
    print("No channel found for query:", query)

{'kind': 'youtube#searchListResponse', 'etag': 'EgVfl1YXiVBWV8-IVTdiOa4hHX4', 'regionCode': 'US', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}, 'items': [{'kind': 'youtube#searchResult', 'etag': 't4psSHWi7XS_dDiepKb1kNFrcNc', 'id': {'kind': 'youtube#channel', 'channelId': 'UCybix9PZoDgcyyt5hNxPLuw'}, 'snippet': {'publishedAt': '2019-04-29T20:45:33Z', 'channelId': 'UCybix9PZoDgcyyt5hNxPLuw', 'title': 'The Unapologetic Man Podcast With Mark Sing', 'description': "In today's world, there are so many forces that make us try to feel guilty for being MEN. This podcast was created to give guys just ...", 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AIdro_lf3oCnp7K0CzTEtU36TynlgF5AzzVCF80PNQ5C0QKOQmc=s88-c-k-c0xffffffff-no-rj-mo'}, 'medium': {'url': 'https://yt3.ggpht.com/ytc/AIdro_lf3oCnp7K0CzTEtU36TynlgF5AzzVCF80PNQ5C0QKOQmc=s240-c-k-c0xffffffff-no-rj-mo'}, 'high': {'url': 'https://yt3.ggpht.com/ytc/AIdro_lf3oCnp7K0CzTEtU36TynlgF5AzzVCF80PNQ5C0QKOQmc=s800-c-k-c0xfffff

### Get video ID's from channel ID

In [164]:

def get_all_video_ids_from_channel(channel_id, max_results=200):
    uploads_playlist_id = "UU" + channel_id[2:]
    video_ids = []
    next_page_token = None

    while True:
        req = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        res = req.execute()
        for item in res['items']:
            video_ids.append(item['contentDetails']['videoId'])
        next_page_token = res.get('nextPageToken')
        return video_ids

channel_id = "UC_x5XG1OV2P6uZZ5FSM9Ttw"
video_ids = get_all_video_ids_from_channel(channel_id)
print(f"Found {len(video_ids)} video IDs.")
print(video_ids)

Found 50 video IDs.
['EmPZx26--7o', 'yvmeHLEQI44', 'zYz2dPKnNa0', 'W3puqnctOh8', 'zUStGLjVw70', 'f7q3amS7uOY', '9OJ6HbJGWH0', 'Gs112JPIsgM', 'K4vXvaRV0dw', '2cFKh6x6KUc', 'UjhpIR79bOU', 'cMx4s-rwodk', 'sS7afkjLc8U', 'QDS3aO2xeJY', 'LaLK_FyZOdc', 'xbTIzTjpOTQ', 'GFQaUd1JF-U', 'YxtLYKHntVs', 'tVh5R1Z1q9k', 'NW0NssvEeDs', '3GYgkcFd_ec', '10deLft7KIE', 'UIntXBP--gI', 'LtRi2F3JRpo', '0qhKmtsfBaw', 'srQpWTIcN_A', 'zgEUfr48PmE', 'jwbG_m-X-gE', 'i3Pc-Z8YzMQ', 'eFXr0DrbcVc', '_cdjPCkdGXU', 'eCLMWJY1RAg', '21_pmVP7gtA', 'Cc4rrw0fee8', '9_GiWxsIwrE', 'qqHFXC2n2Xc', 'Smk_SA8GoTU', 'YW1m2Z6MyTk', 'zcdMZImwlNo', 'MwJjFp6V8PE', 'PvKEHPbZ4-Y', '2YzXNLl1W0Y', 'MUn6W73HffY', 'JMIenSHitQk', 'fXtKyxUKDpc', 'NQO78R6pyp8', 'EysJTNLQVZw', 'B8Fdxdri0aQ', '-K7zY3VubVo', 'VM3b3csBeUc']


In [43]:
# from youtube_transcript_api import YouTubeTranscriptApi
# import json
# transcript = YouTubeTranscriptApi.get_transcript(video_ids[0])
# with open(f"{video_ids[0]}_transcript.json", "w") as f:
#     json.dump(transcript, f, indent=2)

# # DOes not work well, use selenium instead to scrape the transcript

In [72]:
video_urls = [f"https://www.youtube.com/watch?v={vid}" for vid in video_ids]

## Testing with selenium to get video transcript

In [93]:
video_url = video_urls[0]
print(video_url)

https://www.youtube.com/watch?v=uKRtjm2WvB0


In [82]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pyperclip
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
# for url in video_url:
#     driver.get("https://www.youtube-transcript.io/")
#     time.sleep(2) 
#     input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
#     input_box.click()
#     input_box.clear()
#     pyperclip.copy(url)
#     # input_box.send_keys(url)
#     # input_box.send_keys(Keys.RETURN)
#     input_box.send_keys(Keys.COMMAND, 'v')  # On Mac, use Keys.COMMAND, 'v'
#     input_box.send_keys(Keys.RETURN)
#     time.sleep(5)
#     # transcript_div = driver.find_element(By.CSS_SELECTOR, ".transcript-content")
#     # transcript_text = transcript_div.text
#     # print(transcript_text)
#     # copy_button = driver.find_element(By.XPATH, "//button[contains(., 'Copy Transcript')]")
#     # copy_button.click()
#     # time.sleep(1)
#     wait = WebDriverWait(driver, 10)
#     copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/main/div/div[2]/div[2]/div/div/div[1]/button[1]")))
#     copy_button.click()

def get_transcript(url):
    driver.get("https://www.youtube-transcript.io/")
    time.sleep(2) 
    input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
    input_box.click()
    input_box.clear()
    pyperclip.copy(url)
    # input_box.send_keys(url)
    # input_box.send_keys(Keys.RETURN)
    input_box.send_keys(Keys.COMMAND, 'v')  # On Mac, use Keys.COMMAND, 'v'
    input_box.send_keys(Keys.RETURN)
    time.sleep(5)
    # transcript_div = driver.find_element(By.CSS_SELECTOR, ".transcript-content")
    # transcript_text = transcript_div.text
    # print(transcript_text)
    # copy_button = driver.find_element(By.XPATH, "//button[contains(., 'Copy Transcript')]")
    # copy_button.click()
    # time.sleep(1)
    wait = WebDriverWait(driver, 10)
    # copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/main/div/div[2]/div[2]/div/div/div[1]/button[1]")))
    copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
    copy_button.click()

get_transcript(video_url)

In [83]:
print(pyperclip.paste())

What do you do when a girl flakes on you an hour before you're supposed to meet with her? She texts you, "I I I had to stay late with my boss, XYZ." What do you do to that? Okay. If she texts you and she flakes, no reply. And you never text her again until she starts chasing you. You drop her ass. She's on ice. Wait for her to come back. Post pictures of you on Instagram with other girls. Post pictures of you doing cool If she doesn't come back, she's done. Now, this is especially good with iPhones because it'll show that you read it, but you didn't even reply back to it. If she has an Android, you have an iPhone, or she has an iPhone and you have an Android, and there's not a send receipt or a red receipt, then you just put okay. Okay, no big deal. This shows that you don't really give a You're going to go talk to a different


## Build a pipeline to feed this into an AI tool (gemini because it is free) -> convert into trainable metrics for LLama 3

In [87]:
import pyperclip
import os
import requests
from google import genai

# 1. Get the transcript text from clipboard
transcript_text = pyperclip.paste()
# 2. Build the Gemini prompt
llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
<|eot_id|>
"""

gemini_instruction = (
    "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
    "For each, provide a JSON object with the following fields: "
    "question (the user's question or implied question), "
    "context (any relevant background or setup), "
    "answer (the assistant's response), "
    "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
    "Format your output as a JSON list. "
    "Use this template for 'text':\n"
    "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
    "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    "{question}\n"
    "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    "{answer}\n"
    "<|eot_id|>\n"
    "Here is the transcript:\n\n"
    f"{transcript_text}"
)

# 3. (Optional) Send to Gemini API (pseudo-code, as actual API usage may vary)
# url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=" + GEMINI_API_KEY
# headers = {"Content-Type": "application/json"}
# data = {
#     "contents": [
#         {"parts": [{"text": gemini_instruction}]}
#     ]
# }

# response = requests.post(url, headers=headers, json=data)
# result = response.json()

client = genai.Client(api_key=GEMINI_API_KEY)
response = client.models.generate_content(
            model="gemini-2.5-flash" , # or "gpt-3.5-turbo"
            contents=gemini_instruction
        )
res = response.text
print(res)

# 4. (Optional) Save the result to a file
with open("gemini_output.json", "w") as f:
    f.write(str(res))

Python(25894) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


```json
[
  {
    "question": "What do you do when a girl flakes on you an hour before you're supposed to meet?",
    "context": "She texts you, 'I had to stay late with my boss, XYZ.'",
    "answer": "If she texts you and she flakes, the rule is no reply. You never text her again until she starts chasing you. You drop her ass; she's on ice. Wait for her to come back. Post pictures of yourself on Instagram with other girls or doing cool activities. If she doesn't come back, she's done. This tactic works especially well with iPhones, as it shows you read the message but chose not to reply. If read receipts aren't available (e.g., cross-device messaging), then you just text 'okay' or 'Okay, no big deal.' This shows you don't really give a damn and that you're going to go talk to a different girl.",
    "text": "|begin_of_text|><|start_header_id|>system<|end_header_id|>\\nYou are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful ba

### Making an ETL pipeline to continously process each video becuase I am too lazy to copy paste every single time

In [90]:
print(len(video_urls))

200


In [95]:
urls = video_urls[:2]
print(urls)

['https://www.youtube.com/watch?v=uKRtjm2WvB0', 'https://www.youtube.com/watch?v=M31Cu9WlosQ']


In [ ]:
# cleaned = res.strip()
# # Remove starting ```json and ending ```
# if cleaned.startswith("```json"):
#     cleaned = cleaned[len("```json"):].lstrip()
# if cleaned.endswith("```"):
#     cleaned = cleaned[:-3].rstrip()
# with open("output.json", "w") as f:
#     f.write(cleaned)

In [104]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
cnt = 0
for url in video_urls:
    driver.get("https://www.youtube-transcript.io/")
    time.sleep(2) 
    input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
    input_box.click()
    input_box.clear()
    pyperclip.copy(url)
    # input_box.send_keys(Keys.COMMAND, 'v')  
    input_box.send_keys(url)  # Directly type the URL
    input_box.send_keys(Keys.RETURN)
    time.sleep(5)
    wait = WebDriverWait(driver, 10)
    driver.save_screenshot("debug_headless.png")
    wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
    copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
    copy_button.click()
    transcript_text = pyperclip.paste()
    # 2. Build the Gemini prompt
    llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
    <|eot_id|>
    """

    gemini_instruction = (
        "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
        "For each, provide a JSON object with the following fields: "
        "question (the user's question or implied question), "
        "context (any relevant background or setup), "
        "answer (the assistant's response), "
        "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
        "Format your output as a JSON list. "
        "Use this template for 'text':\n"
        "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
        "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
        "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
        "{question}\n"
        "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        "{answer}\n"
        "<|eot_id|>\n"
        "Here is the transcript:\n\n"
        f"{transcript_text}"
    )
    client = genai.Client(api_key=GEMINI_API_KEY)
    response = client.models.generate_content(
                model="gemini-2.5-flash" , # or "gpt-3.5-turbo"
                contents=gemini_instruction
            )
    res = response.text
    cleaned = res.strip()
    # Remove starting ```json and ending ```
    if cleaned.startswith("```json"):
        cleaned = cleaned[len("```json"):].lstrip()
    if cleaned.endswith("```"):
        cleaned = cleaned[:-3].rstrip()
    with open(f"gemini_output{cnt}.json", "w") as f:
        f.write(cleaned)
    cnt += 1
   


Python(30722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31092) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31109) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31116) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31139) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31140) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31161) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31164) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31184) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31185) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(31200) Malloc

AttributeError: 'NoneType' object has no attribute 'strip'

In [105]:
print(type(res))

<class 'NoneType'>


In [107]:
print(transcript_text)

https://www.youtube.com/watch?v=_TZKYs2SpCo


In [109]:
print(video_urls[91])

https://www.youtube.com/watch?v=_TZKYs2SpCo


In [129]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.youtube-transcript.io/")
url = "https://www.youtube.com/watch?v=ss5g1m2GhMM"
time.sleep(2)
input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
input_box.click()
input_box.clear()
input_box.send_keys(url)
input_box.send_keys(Keys.RETURN)
time.sleep(5)
wait = WebDriverWait(driver, 10)
driver.save_screenshot("debug_headless.png")
wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
copy_button.click()
transcript_text = pyperclip.paste()
llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
<|eot_id|>
"""
gemini_instruction = (
    "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
    "For each, provide a JSON object with the following fields: "
    "question (the user's question or implied question), "
    "context (any relevant background or setup), "
    "answer (the assistant's response), "
    "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
    "Format your output as a JSON list. "
    "Use this template for 'text':\n"
    "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
    "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    "{question}\n"
    "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    "{answer}\n"
    "<|eot_id|>\n"
    "Here is the transcript:\n\n"
    f"{transcript_text}"
)
print(transcript_text)
client = genai.Client(api_key=GEMINI_API_KEY)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=gemini_instruction
)
res = response.text
cleaned = res.strip()
if cleaned.startswith("```json"):
    cleaned = cleaned[len("```json"):].lstrip()
if cleaned.endswith("```"):
    cleaned = cleaned[:-3].rstrip()
with open(f"data/gemini_output122.json", "w") as f:
    f.write(cleaned)

Python(41263) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(41632) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(41663) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


https://www.youtube.com/watch?v=ss5g1m2GhMM


In [128]:
print(transcript_text)

https://www.youtube.com/watch?v=ss5g1m2GhMM


In [113]:
for url in video_urls[91:]:
    print(url)

https://www.youtube.com/watch?v=_TZKYs2SpCo
https://www.youtube.com/watch?v=4nEAB0syElI
https://www.youtube.com/watch?v=uJndSv4ceiY
https://www.youtube.com/watch?v=XO0Grm5vIEg
https://www.youtube.com/watch?v=V7rJDgeNe2c
https://www.youtube.com/watch?v=-1kw3iKPENc
https://www.youtube.com/watch?v=kvwetHClQeI
https://www.youtube.com/watch?v=SnVs1gEZduo
https://www.youtube.com/watch?v=MYT1Wq2FOxY
https://www.youtube.com/watch?v=98QH9R1MtiU
https://www.youtube.com/watch?v=7JZ68eSSW7Q
https://www.youtube.com/watch?v=GSjcXTd1oHA
https://www.youtube.com/watch?v=gNeR6hsVqCo
https://www.youtube.com/watch?v=W3HmzCvioUg
https://www.youtube.com/watch?v=AIBFd85Jkxc
https://www.youtube.com/watch?v=7igIYiSEB-4
https://www.youtube.com/watch?v=HxZSmbkdV1A
https://www.youtube.com/watch?v=7pBv9fAa_4M
https://www.youtube.com/watch?v=SdkbFeJZTmg
https://www.youtube.com/watch?v=xvIyaMo5k3M
https://www.youtube.com/watch?v=7_BO6vt7XvI
https://www.youtube.com/watch?v=K-BMy6xmf6w
https://www.youtube.com/watch?v=

## Ran into erro at 91st, resuming from 92

In [126]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
cnt = 122
failed_indices = []  # Track failed attempts

for url in video_urls[122]:
    try:
        driver.get("https://www.youtube-transcript.io/")
        time.sleep(2)
        input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
        input_box.click()
        input_box.clear()
        # pyperclip.copy(url)
        input_box.send_keys(url)
        input_box.send_keys(Keys.RETURN)
        wait = WebDriverWait(driver, 10)
        # driver.save_screenshot("debug_headless.png")
        # wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
        copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
        copy_button.click()
        transcript_text = pyperclip.paste()
        llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
<|eot_id|>
"""
        gemini_instruction = (
            "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
            "For each, provide a JSON object with the following fields: "
            "question (the user's question or implied question), "
            "context (any relevant background or setup), "
            "answer (the assistant's response), "
            "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
            "Format your output as a JSON list. "
            "Use this template for 'text':\n"
            "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
            "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
            "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
            "{question}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
            "{answer}\n"
            "<|eot_id|>\n"
            "Here is the transcript:\n\n"
            f"{transcript_text}"
        )
        client = genai.Client(api_key=GEMINI_API_KEY)
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=gemini_instruction
        )
        res = response.text
        
        # Check if res is None and skip if so
        if res is None:
            print(f"Skipping video {cnt}: res is None")
            failed_indices.append(cnt)
            cnt += 1
            continue
            
        cleaned = res.strip()
        if cleaned.startswith("```json"):
            cleaned = cleaned[len("```json"):].lstrip()
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3].rstrip()
        with open(f"data/gemini_output{cnt}.json", "w") as f:
            f.write(cleaned)
        cnt += 1
        
    except Exception as e:
        print(f"Error processing video {cnt}: {str(e)}")
        failed_indices.append(cnt)
        cnt += 1
        continue

# Print summary of failed attempts
print(f"Failed indices: {failed_indices}")
print(f"Total failed: {len(failed_indices)}")

Python(40160) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40529) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40547) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Error processing video 122: Message: 
Stacktrace:
0   chromedriver                        0x0000000101134e6c cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x000000010112cd74 cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x0000000100c7e3ec cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100cc5544 cxxbridge1$string$len + 381808
4   chromedriver                        0x0000000100d06934 cxxbridge1$string$len + 649056
5   chromedriver                        0x0000000100cb9834 cxxbridge1$string$len + 333408
6   chromedriver                        0x00000001010f7f88 cxxbridge1$str$ptr + 2473268
7   chromedriver                        0x00000001010fb1f4 cxxbridge1$str$ptr + 2486176
8   chromedriver                        0x00000001010d99d0 cxxbridge1$str$ptr + 2348924
9   chromedriver                        0x00000001010fbab0 cxxbridge1$str$ptr + 2488412
10  chromedriver                        0x00000001010caa60 cxxb

Python(40575) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Error processing video 123: Message: 
Stacktrace:
0   chromedriver                        0x0000000101134e6c cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x000000010112cd74 cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x0000000100c7e3ec cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100cc5544 cxxbridge1$string$len + 381808
4   chromedriver                        0x0000000100d06934 cxxbridge1$string$len + 649056
5   chromedriver                        0x0000000100cb9834 cxxbridge1$string$len + 333408
6   chromedriver                        0x00000001010f7f88 cxxbridge1$str$ptr + 2473268
7   chromedriver                        0x00000001010fb1f4 cxxbridge1$str$ptr + 2486176
8   chromedriver                        0x00000001010d99d0 cxxbridge1$str$ptr + 2348924
9   chromedriver                        0x00000001010fbab0 cxxbridge1$str$ptr + 2488412
10  chromedriver                        0x00000001010caa60 cxxb

Python(40578) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Error processing video 124: Message: 
Stacktrace:
0   chromedriver                        0x0000000101134e6c cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x000000010112cd74 cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x0000000100c7e3ec cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100cc5544 cxxbridge1$string$len + 381808
4   chromedriver                        0x0000000100d06934 cxxbridge1$string$len + 649056
5   chromedriver                        0x0000000100cb9834 cxxbridge1$string$len + 333408
6   chromedriver                        0x00000001010f7f88 cxxbridge1$str$ptr + 2473268
7   chromedriver                        0x00000001010fb1f4 cxxbridge1$str$ptr + 2486176
8   chromedriver                        0x00000001010d99d0 cxxbridge1$str$ptr + 2348924
9   chromedriver                        0x00000001010fbab0 cxxbridge1$str$ptr + 2488412
10  chromedriver                        0x00000001010caa60 cxxb

KeyboardInterrupt: 

In [121]:
print(cnt)
print(len(transcript_text))
print(transcript_text)

121
43
https://www.youtube.com/watch?v=ss5g1m2GhMM


In [118]:
failed_incices_track = []
for fl in failed_indices:
    failed_incices_track.append(fl)

print(failed_incices_track)

[96, 101, 102, 114, 115]


### Added more robust error handling, such as the text not being transcribed or empty and a seprate folder for the files

In [125]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
cnt = 121
failed_indices = []  # Track failed attempts

for url in video_urls[121:]:
    try:
        driver.get("https://www.youtube-transcript.io/")
        time.sleep(2)
        input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
        input_box.click()
        input_box.clear()
        pyperclip.copy(url)
        input_box.send_keys(url)
        input_box.send_keys(Keys.RETURN)
        time.sleep(5)
        wait = WebDriverWait(driver, 10)
        driver.save_screenshot("debug_headless.png")
        wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
        copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
        copy_button.click()
        transcript_text = pyperclip.paste()
        print(transcript_text)
        # Check if transcript_text is valid
        if not transcript_text or len(transcript_text.strip()) == 0:
            print(f"Skipping video {cnt}: Empty transcript")
            failed_indices.append(cnt)
            cnt += 1
            continue
            
        # Check if transcript is a URL/link (not actual transcript)
        if transcript_text.startswith("http") or "youtube.com" in transcript_text.lower():
            print(f"Skipping video {cnt}: Transcript is a URL/link")
            failed_indices.append(cnt)
            cnt += 1
            continue
            
        # Check if transcript has sufficient content (more than 200 words)
        word_count = len(transcript_text.split())
        if word_count < 200:
            print(f"Skipping video {cnt}: Transcript too short ({word_count} words)")
            failed_indices.append(cnt)
            cnt += 1
            continue
        
        llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
<|eot_id|>
"""
        gemini_instruction = (
            "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
            "For each, provide a JSON object with the following fields: "
            "question (the user's question or implied question), "
            "context (any relevant background or setup), "
            "answer (the assistant's response), "
            "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
            "Format your output as a JSON list. "
            "Use this template for 'text':\n"
            "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
            "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
            "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
            "{question}\n"
            "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
            "{answer}\n"
            "<|eot_id|>\n"
            "Here is the transcript:\n\n"
            f"{transcript_text}"
        )
        client = genai.Client(api_key=GEMINI_API_KEY)
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=gemini_instruction
        )
        res = response.text
        
        # Check if res is None and skip if so
        if res is None:
            print(f"Skipping video {cnt}: res is None")
            failed_indices.append(cnt)
            cnt += 1
            continue
            
        cleaned = res.strip()
        if cleaned.startswith("```json"):
            cleaned = cleaned[len("```json"):].lstrip()
        if cleaned.endswith("```"):
            cleaned = cleaned[:-3].rstrip()
        with open(f"data/gemini_output{cnt}.json", "w") as f:
            f.write(cleaned)
        cnt += 1
        
    except Exception as e:
        print(f"Error processing video {cnt}: {str(e)}")
        failed_indices.append(cnt)
        cnt += 1
        continue

# Print summary of failed attempts
print(f"Failed indices: {failed_indices}")
print(f"Total failed: {len(failed_indices)}")

Python(39709) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40078) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40096) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40100) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


https://www.youtube.com/watch?v=ss5g1m2GhMM
Skipping video 121: Transcript is a URL/link


Python(40111) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


KeyboardInterrupt: 

### The data collected is not the actual transcript data for some files, as the copy/paste is not working properly from the website

# Try 3

In [155]:
import subprocess
chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
cnt = 0
failed_indices = []  # Track failed attempts
failed_urls = []

def clear_clipboard():
    subprocess.run(['pbcopy'], input='', text=True)

def get_clipboard_content():
    result = subprocess.run(['pbpaste'], capture_output=True, text=True)
    return result.stdout
for url in video_urls[:1]:
        try:
            driver.get("https://www.youtube-transcript.io/")
            time.sleep(2)
            input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
            input_box.click()
            input_box.clear()
            input_box.send_keys(url)
            input_box.send_keys(Keys.RETURN)
            wait = WebDriverWait(driver, 10)
            # driver.save_screenshot("debug_headless.png")
            # wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
            clear_clipboard()
            time.sleep(0.5) 
            copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
            copy_button.click()
            time.sleep(1)
            
            # Get clipboard content using system command
            transcript_text = get_clipboard_content()        # robust error handling
            if not transcript_text or len(transcript_text.strip()) == 0:
                print(f"Skipping video {cnt}: Empty transcript")
                failed_indices.append(cnt)
                failed_urls.append(url)
                cnt += 1
                continue
                
            # Check if transcript is a URL/link (not actual transcript)
            if transcript_text.startswith("http") or "youtube.com" in transcript_text.lower():
                print(f"Skipping video {cnt}: Transcript is a URL/link")
                failed_indices.append(cnt)
                failed_urls.append(url)
                print(transcript_text)
                cnt += 1
                continue
                
            # Check if transcript has sufficient content (more than 200 words)
            word_count = len(transcript_text.split())
            if word_count < 200:
                print(f"Skipping video {cnt}: Transcript too short ({word_count} words)")
                failed_indices.append(cnt)
                failed_urls.append(url) # can rerun later
                print(transcript_text)

                cnt += 1
                continue
            llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
    <|eot_id|>
    """
            gemini_instruction = (
                "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
                "For each, provide a JSON object with the following fields: "
                "question (the user's question or implied question), "
                "context (any relevant background or setup), "
                "answer (the assistant's response), "
                "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
                "Format your output as a JSON list. "
                "Use this template for 'text':\n"
                "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
                "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
                "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
                "{question}\n"
                "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
                "{answer}\n"
                "<|eot_id|>\n"
                "Here is the transcript:\n\n"
                f"{transcript_text}"
            )
            client = genai.Client(api_key=GEMINI_API_KEY)
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=gemini_instruction
            )
            res = response.text
            
            # Check if res is None and skip if so
            if res is None:
                print(f"Skipping video {cnt}: res is None")
                failed_indices.append(cnt)
                failed_urls.append(url)
                cnt += 1
                continue
                
            cleaned = res.strip()
            if cleaned.startswith("```json"):
                cleaned = cleaned[len("```json"):].lstrip()
            if cleaned.endswith("```"):
                cleaned = cleaned[:-3].rstrip()
            with open(f"data/gemini_output{cnt}.json", "w") as f:
                f.write(cleaned)
            with open(f"data/transcript_output{cnt}.json", "w") as f:
                f.write(transcript_text)
            cnt += 1
            
        except Exception as e:
            print(f"Error processing video {cnt}: {str(e)}")
            failed_indices.append(cnt)
            failed_urls.append(url)
            cnt += 1
            continue

Python(49066) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(49435) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(49452) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Skipping video 0: Transcript too short (171 words)
What do you do when a girl flakes on you an hour before you're supposed to meet with her? She texts you, "I I I had to stay late with my boss, XYZ." What do you do to that? Okay. If she texts you and she flakes, no reply. And you never text her again until she starts chasing you. You drop her ass. She's on ice. Wait for her to come back. Post pictures of you on Instagram with other girls. Post pictures of you doing cool If she doesn't come back, she's done. Now, this is especially good with iPhones because it'll show that you read it, but you didn't even reply back to it. If she has an Android, you have an iPhone, or she has an iPhone and you have an Android, and there's not a send receipt or a red receipt, then you just put okay. Okay, no big deal. This shows that you don't really give a You're going to go talk to a different


Python(49465) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [147]:
s = pyperclip.paste()
print(s)

Python(45504) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [143]:
print(transcript_text)

women are all attracted to the same damn thing what is that thing obviously confidence self-belief self-esteem good body language leader of men pre-selection meaning you have other women in your life and you're already attractive to women they're attracted to ambition every single woman whether you're in Africa Japan or Italy is attracted to a man who's ambitious who's passionate who's trying to make something of himself who can control the fr who can lead her who can make her laugh


In [138]:
import subprocess
result = subprocess.run(['pbpaste'], capture_output=True, text=True)
print(result.stdout)

women are all attracted to the same damn thing what is that thing obviously confidence self-belief self-esteem good body language leader of men pre-selection meaning you have other women in your life and you're already attractive to women they're attracted to ambition every single woman whether you're in Africa Japan or Italy is attracted to a man who's ambitious who's passionate who's trying to make something of himself who can control the fr who can lead her who can make her laugh


Python(43624) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


### Adding head mode instead of headless mode, as that is preventing pyperclip from copying text properly

In [159]:
"""
This script is used to get the transcript of the youtube videos and save it to a json file.
It also saves the failed urls to a file.
-- added error handling for empty transcript, url link, and short transcript
-- added transcript saving to a file
-- added failed urls to a file
-- added failed indices to a file
-- added head mode
--  data created before this is labelled synthetic data
-- constant checks to ensure data is from actual podcast and not a url link
-- constant checks to ensure data is not empty
-- constant checks to ensure data is not too short
-- constant checks to ensure data is not too long
-- constant checks to ensure data is not too short
-- using subprocess to clear clipboard and get clipboard content
-- debuged to find some videos are shorter so changed the threshold to 170 words
"""

chrome_options = Options()
chrome_options.add_argument("--incognito")
all_results = []
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)
cnt = 17
failed_indices = []  # Track failed attempts
failed_urls = []

def clear_clipboard():
    subprocess.run(['pbcopy'], input='', text=True)

def get_clipboard_content():
    result = subprocess.run(['pbpaste'], capture_output=True, text=True)
    return result.stdout

try: 
    for url in video_urls[17:]:
        try:
            driver.get("https://www.youtube-transcript.io/")
            time.sleep(2)
            input_box = driver.find_element(By.CSS_SELECTOR, "input[type='url']")
            input_box.click()
            input_box.clear()
            input_box.send_keys(url)
            input_box.send_keys(Keys.RETURN)
            wait = WebDriverWait(driver, 10)
            # driver.save_screenshot("debug_headless.png")
            # wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
            clear_clipboard()
            time.sleep(0.5) 
            copy_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Copy Transcript')]")))
            copy_button.click()
            time.sleep(1)
            
            # Get clipboard content using system command
            transcript_text = get_clipboard_content()        # robust error handling
            if not transcript_text or len(transcript_text.strip()) == 0:
                print(f"Skipping video {cnt}: Empty transcript")
                failed_indices.append(cnt)
                failed_urls.append(url)
                cnt += 1
                continue
                
            # Check if transcript is a URL/link (not actual transcript)
            if transcript_text.startswith("http") or "youtube.com" in transcript_text.lower():
                print(f"Skipping video {cnt}: Transcript is a URL/link")
                failed_indices.append(cnt)
                failed_urls.append(url)
                print(transcript_text)
                cnt += 1
                continue
                
            # Check if transcript has sufficient content (more than 200 words)
            word_count = len(transcript_text.split())
            if word_count < 170:
                print(f"Skipping video {cnt}: Transcript too short ({word_count} words)")
                failed_indices.append(cnt)
                failed_urls.append(url) # can rerun later
                print(transcript_text)

                cnt += 1
                continue
            llama_template = """|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.
    <|eot_id|>
    """
            gemini_instruction = (
                "Given the following podcast transcript, extract at least 15 Q&A pairs and at most 30 Q&A pairs. "
                "For each, provide a JSON object with the following fields: "
                "question (the user's question or implied question), "
                "context (any relevant background or setup), "
                "answer (the assistant's response), "
                "and text (a string combining the question, context, and answer, formatted for Llama training as shown below). "
                "Format your output as a JSON list. "
                "Use this template for 'text':\n"
                "|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
                "You are Mark Singh, a dating coach who gives practical advice to young men with an unapologetic, witty, and playful banter style.\n"
                "<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
                "{question}\n"
                "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
                "{answer}\n"
                "<|eot_id|>\n"
                "Here is the transcript:\n\n"
                f"{transcript_text}"
            )
            client = genai.Client(api_key=GEMINI_API_KEY)
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=gemini_instruction
            )
            res = response.text
            
            # Check if res is None and skip if so
            if res is None:
                print(f"Skipping video {cnt}: res is None")
                failed_indices.append(cnt)
                failed_urls.append(url)
                cnt += 1
                continue
                
            cleaned = res.strip()
            if cleaned.startswith("```json"):
                cleaned = cleaned[len("```json"):].lstrip()
            if cleaned.endswith("```"):
                cleaned = cleaned[:-3].rstrip()
            with open(f"data/gemini_output{cnt}.json", "w") as f:
                f.write(cleaned)
            with open(f"data/transcript_output{cnt}.txt", "w") as f:
                f.write(transcript_text)
            cnt += 1
            
        except Exception as e:
            print(f"Error processing video {cnt}: {str(e)}")
            failed_indices.append(cnt)
            failed_urls.append(url)
            cnt += 1
            continue
finally:
    driver.quit()

# Print summary of failed attempts
print(f"Failed indices: {failed_indices}")
print(f"Total failed: {len(failed_indices)}")

Python(51264) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51633) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51647) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51654) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51680) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51682) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51694) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51695) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51709) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51711) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51717) Malloc

Skipping video 31: Transcript too short (157 words)
if you want to get a nine or a 10 brother you almost always have to have girls in your life to begin with and you almost always have to have that nine or 10 see you with other girls see that other girls want to be around you and more importantly see that those girls are attracted to you and you have girls around you already they will become more attracted to you the hotter the girls you have around you the more attractive you become if I take you and I send you into a nightclub all by yourself you're going to have a certain level of attractiveness if I took that exact same you we don't change anything about you other than I put two hot girls on your arm your attractiveness to women will literally 10x yeah the bigger the cap the bigger the pillar won't stop till these busters my cast was


Python(51908) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51909) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51922) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51923) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51958) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51961) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51982) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51983) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Skipping video 35: Transcript too short (149 words)
unless you're seriously paranoid unless you're insecure and you're super needy and you're just jealous all the time if you think she's cheating bro she probably is and she's just being real private and she's just being suspicious and you see her flirting with other dudes and perhaps even you know about her reputation she's probably cheating on you bro I mean she's being secretive she's lying she's keeping her phone from you she admits it Without Really admitting it she's happy suddenly but it has nothing to do with you and or she starts being mean to you well I'm going to make him the enemy I am going to convince myself that he is a piece of he is a bad husband and I'm going to start being mean to him it's because she feels guilty about cheating so be careful who you choose and of course


Python(51984) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(51985) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52003) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52004) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52027) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52029) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52051) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52052) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(52064) Malloc

...

Skipping video 194: Transcript too short (79 words)
women are programmed to become attracted to those attributes to those personality signals that this guy's an alpha male and he has high survivability genes you don't have to be successful now in order to show that you can be successful later and that's all she cares about does he have the potential to become an amazing success yes he does men are concerned about a woman's past women are concerned about a man's future think about that one


Python(54068) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54070) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54091) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54093) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Skipping video 196: Transcript too short (153 words)
here's my Golden Rule when it comes to attracting women you want to always ask yourself what would I do if I were dating four girls I actually want you to pretend you have a bracelet on that says what would I do if I were dating four girls why is that because when you run everything through that filter you always behave correctly you don't care what happens you're not outcome independent if she calls you back fine if she doesn't that's fine too you don't give a because you're dating four girls then you behave correctly and they do get attracted to you so even if you're dating nobody or just one girl or two girls always ask yourself in this situation what would I do if I were dating four girls when you run it through that filter you will always make the correct decision when it comes to women


Python(54097) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54098) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54113) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54114) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54132) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54133) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Skipping video 199: Transcript too short (125 words)
two reasons why women cheat and how to prevent it in the future the first reason is you're not masculine enough you're not man enough you're not ambitious enough you're not exciting enough and she wants to go find that another guy the second reason is because you're not listening to her you're not paying attention to her you're not putting trust and connection into the relationship and she feels like you're just an empty shell who she can't relate to so she tries to go find that in a different guy if you can lead the frame be that exciting dude but at the same time be her Confidant in her man that she can talk to you'll never get cheated on again yeah
Failed indices: [31, 35, 41, 42, 53, 54, 57, 58, 62, 68, 69, 71, 73, 75, 76, 77, 79, 80, 85, 87, 88, 89, 93, 94, 95, 97, 98, 100, 101, 103, 104, 105, 106, 108, 109, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 145, 148, 149, 162, 163, 164, 165, 172, 173, 176, 177, 179, 18

In [160]:
print(failed_indices)

[31, 35, 41, 42, 53, 54, 57, 58, 62, 68, 69, 71, 73, 75, 76, 77, 79, 80, 85, 87, 88, 89, 93, 94, 95, 97, 98, 100, 101, 103, 104, 105, 106, 108, 109, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 145, 148, 149, 162, 163, 164, 165, 172, 173, 176, 177, 179, 180, 182, 186, 188, 189, 194, 196, 199]


In [161]:
print(failed_urls)

['https://www.youtube.com/watch?v=6h_dDtI3SFk', 'https://www.youtube.com/watch?v=ueyGh-vfVVU', 'https://www.youtube.com/watch?v=p2rJ0tjHLao', 'https://www.youtube.com/watch?v=jYYZSybOzXw', 'https://www.youtube.com/watch?v=5Xms1VXVNA0', 'https://www.youtube.com/watch?v=w3zgh6nawJ4', 'https://www.youtube.com/watch?v=pWNDlUCV2Kg', 'https://www.youtube.com/watch?v=vqoTzz_iApI', 'https://www.youtube.com/watch?v=OhGN9bkJO5s', 'https://www.youtube.com/watch?v=jha5p5ivBt8', 'https://www.youtube.com/watch?v=735AKwusajw', 'https://www.youtube.com/watch?v=bdAnEluT70U', 'https://www.youtube.com/watch?v=13ddeiOIe4w', 'https://www.youtube.com/watch?v=TxwTPK5Jssg', 'https://www.youtube.com/watch?v=AktQTKHSAOQ', 'https://www.youtube.com/watch?v=UsYkaRacB1Y', 'https://www.youtube.com/watch?v=p8G8L4OCgE4', 'https://www.youtube.com/watch?v=9ldFL--zUXE', 'https://www.youtube.com/watch?v=aN4vzOQbNP4', 'https://www.youtube.com/watch?v=_tCMjkbZ67c', 'https://www.youtube.com/watch?v=KFGeJKTM2m0', 'https://www